In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from string import punctuation
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import ngrams
from itertools import chain
from wordcloud import WordCloud
import sense2vec
from sense2vec import Sense2VecComponent
import spacy
from spacy import displacy
from spacy.util import minibatch, compounding
from sense2vec import Sense2Vec
import re
import pickle

In [2]:
trainingSet = pd.read_csv('train.csv')
test_index = pd.read_csv("test.csv")['Id'].tolist()
trainingSet = trainingSet.dropna()
trainingSet = trainingSet.set_index('Id')

In [3]:
print(trainingSet['Summary'])

Id
0                                  good version of a classic
1                                     Good but not as moving
2                      Winkler's Performance was ok at best!
3               It's an enjoyable twist on the classic story
4                                           Best Scrooge yet
                                 ...                        
1697526     Worth the wait, but worth the asking price, too?
1697529    HOLY BAT-BOXSET, BATMAN... I never thought thi...
1697530    prayers have been answered because batman 60s ...
1697531                                          can't Wait!
1697532    The Price is Insane? People Really Need to Wak...
Name: Summary, Length: 1397461, dtype: object


In [3]:
fulltrainingSet = pd.read_csv('train.csv')
x = fulltrainingSet['Summary'][fulltrainingSet.Summary != fulltrainingSet.Summary]
y = fulltrainingSet['Text'][fulltrainingSet.Text != fulltrainingSet.Text]

In [4]:
x = np.array(x.index).tolist()
y = np.array(y.index).tolist()

In [5]:
print(x)
s = fulltrainingSet['Text'].iloc[x]

[11603, 84167, 100155, 118232, 128725, 158752, 236956, 266209, 526382, 577522, 640539, 819994, 965397, 1009122, 1269960, 1305629, 1367464, 1391265, 1393501, 1463237, 1496024, 1538579, 1538842, 1588511, 1646534]


In [6]:
from jieba import analyse
tfidf = analyse.extract_tags
sum1 = []
for i in s:
    sum1.append(" ".join(tfidf(i)))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/98/nz1_948x12d7bjtk2jwm41cr0000gn/T/jieba.cache
Loading model cost 0.617 seconds.
Prefix dict has been built successfully.


In [7]:
count = 0
for i in x:
    fulltrainingSet['Summary'].iloc[i] = sum1[count]
    count += 1
print(fulltrainingSet['Summary'].iloc[x])



/Users/lw/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


11603      year am Excellent course able watch several ti...
84167      was like film me were story effects feel while...
100155     did like deleted before finished What more say...
118232     had previously read book believe enjoyed movie...
128725     Gift Purchased friend No idea She says marvelo...
158752     people was time about movies so re just ... be...
236956      MOVIE WATCHED YEARS AGO LOVED WELL NOW BEST KIND
266209     same because release versions films Lucas he G...
526382     movie had seen television lot times wanted get...
577522     was Their too much foolishness enough serious ...
640539     was good DVD had no problems liked after film ...
819994     show when CSI acting total did TV lacks many t...
965397     was film Harry off but there PG 13 see scenes ...
1009122    would me she out crying Oh film somebody befor...
1269960    didn put my cart don know how got there Just r...
1305629    Just wanted vote star bring rating down movie ...
1367464    anime bamboo 

In [8]:
hn = fulltrainingSet['HelpfulnessNumerator']
hd = fulltrainingSet['HelpfulnessDenominator']

In [9]:
z = hn/hd
z[z != z] = 0
fulltrainingSet['Helpfulness'] = z

In [10]:
for i in y:
    fulltrainingSet['Text'].iloc[i] = fulltrainingSet['Summary'].iloc[i]
print(fulltrainingSet['Text'].iloc[y])

5209       I Think My Hard Drive Was Too Big To Take This...
13114                          A Kid's review What the Heck!
23314      This review section is sooo confusing it isn't...
32582                  At Last! Shawshank Arrives on Blu-Ray
82557                               Overpriced but it's good
                                 ...                        
1589648                               A Must-Have Collection
1596058                    A fun movie with good car scenes.
1614571    Digging Deeper into Disney: The Hunchback of N...
1641615                Digitally Remastered? In a Pig's Eye!
1696677    Review of the Bluray - Five star series loses ...
Name: Text, Length: 62, dtype: object


In [11]:
fulltrainingSet = fulltrainingSet[['Id','ProductId','UserId','Score','Time','Summary','Text','Helpfulness']]

In [12]:
fulltrainingSet.to_pickle('modifiedtrainingSet.pickle')